<a href="https://colab.research.google.com/github/eddieward21/transfer-learning/blob/main/InceptionV3_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
import os
import urllib.request
import random
import zipfile
from shutil import copyfile

#imports
from tensorflow.keras.optimizers import RMSprop #Root Mean Squared Propagation optimization algorithm
from tensorflow.keras.applications.inception_v3 import InceptionV3 #the model we will be "transferring learning" from
from tensorflow.keras import Model #starting point
from tensorflow.keras import layers #builds layers of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator #creates duplicates of our images that are flipped and rotated


#extract zip files from the url into our /tmp/ directory.
file_url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
file_name = "catsdogs.zip"
download_dir = "/tmp/"
urllib.request.urlretrieve(file_url, file_name)
zipObject = zipfile.ZipFile(file_name, 'r')
zipObject.extractall()
zipObject.close()

In [5]:
print("number of cat images: ", len(os.listdir("/content/PetImages/Cat")))
print("number of dog images: ", len(os.listdir("/content/PetImages/Dog")))


number of cat images:  12501
number of dog images:  12501


In [6]:
try:
    os.mkdir('/content/cats-v-dogs')
    os.mkdir('/content/cats-v-dogs/training')
    os.mkdir('/content/cats-v-dogs/testing')
    os.mkdir('/content/cats-v-dogs/training/cats')
    os.mkdir('/content/cats-v-dogs/training/dogs')
    os.mkdir('/content/cats-v-dogs/testing/cats')
    os.mkdir('/content/cats-v-dogs/testing/dogs')
    print("Success")
except OSError:
    pass

Success


In [7]:
import random
from shutil import copyfile
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/content/PetImages/Cat/"
TRAINING_CATS_DIR = "/content/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/content/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/content/PetImages/Dog/"
TRAINING_DOGS_DIR = "/content/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/content/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [8]:
print("Number of training dog images: ", len(os.listdir(TRAINING_DOGS_DIR)))
print("Number of training cat images: ", len(os.listdir(TRAINING_CATS_DIR)))
print("Number of testing dog images: ", len(os.listdir(TESTING_DOGS_DIR)))
print("Number of testing dog images: ", len(os.listdir(TESTING_CATS_DIR)))

Number of training dog images:  11250
Number of training cat images:  11250
Number of testing dog images:  1250
Number of testing dog images:  1250


In [9]:
TRAINING_DIR = "/content/cats-v-dogs/training/"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range= 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
    )

train_datagen.flow_from_directory(TRAINING_DIR,
                                  batch_size = 100,
                                  target_size = (150,150),
                                  class_mode = 'binary')

VALIDATION_DIR = "/content/cats-v-dogs/testing/"

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size = 100,
    class_mode = "binary",
    target_size = (150,150)
)



Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [10]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(
    input_shape = (150,150,3),
    weights = None,
    include_top = False
    )

In [11]:
pre_trained_model.load_weights(weights_file)

In [12]:
last_layer = pre_trained_model.get_layer("mixed7")
print(last_layer.output_shape)
output= last_layer.output
print(output)

(None, 7, 7, 768)
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
